This notebook just shows you how to run an agent for testing
- Supposedly there is a json output somewhere, and I would like to find that too, but haven't yet!  Please put in comments if you know where it is.
- The first cell sets up an agent that only goes south
- The other cells set up an environment and run 4 instances of that agent against each other.
- That creates a video that shows all the agents going south.

This is another super useful link for beginners found by James: https://www.kaggle.com/antimatter543/hungry-geese-beginners-testing-not-done-abandone
it has been abandoned but has good info!

In [ ]:
%%writefile submission.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
import numpy as np
from typing import *    
from random import choice, sample

last_action = None

def translate(position: int, direction: Action, columns: int, rows: int) -> int:
    row, column = row_col(position, columns)
    row_offset, column_offset = direction.to_row_col()
    row = (row + row_offset) % rows
    column = (column + column_offset) % columns
    return row * columns + column

def min_distance(position: int, food: List[int], columns: int):
    row, column = row_col(position, columns)
    return min(
        abs(row - food_row) + abs(column - food_column)
        for food_position in food
        for food_row, food_column in [row_col(food_position, columns)]
    )

def adjacent_positions(position: int, columns: int, rows: int) -> List[int]:
    return [
        translate(position, action, columns, rows)
        for action in Action
    ]

def agent(observation, configuration):
    global last_action
    rows, columns = configuration.rows, configuration.columns

    food = observation.food
    geese = observation.geese
    opponents = [
        goose
        for index, goose in enumerate(geese)
        if index != observation.index and len(goose) > 0
    ]

    # Don't move adjacent to any heads
    head_adjacent_positions = {
        opponent_head_adjacent
        for opponent in opponents
        for opponent_head in [opponent[0]]
        for opponent_head_adjacent in adjacent_positions(opponent_head, columns, rows)
    }
    # Don't move into any bodies
    bodies = {position for goose in geese for position in goose}

    # Move to the closest food
    position = geese[observation.index][0]
    actions = {
        action: min_distance(new_position, food, columns)
        for action in Action
        for new_position in [translate(position, action, columns, rows)]
        if (
            new_position not in head_adjacent_positions and
            new_position not in bodies and
            (last_action is None or action != last_action.opposite())
        )
    }
    if any(actions):
        action = min(actions, key=actions.get)
    else:
        # bodies without tails
        bodies = []
        for goose in geese:
            if len(goose) > 1:
                bodies.extend(goose[:-1])
        # check for any empty, but not opposite
        found_empty = None
        for action1 in Action:
            if action1 != last_action.opposite():
                cell = translate(position, action1, columns, rows)
                if cell not in bodies:
                    found_empty = action1
                    break
        if found_empty is None:
            action = choice([action for action in Action])
        else:
            action = found_empty
    last_action = action
    return action.name

In [ ]:
# from kaggle_environments import evaluate, make
# env = make("hungry_geese", debug=True)

In [ ]:
# env.reset()
# env.run(['submission.py', 'submission.py', 'submission.py', 'submission.py'])
# env.render(mode="ipython", width=800, height=700)